In [1]:
import urllib.request
import random
import time
from lxml import html
import os
import shutil
import random
import spacy
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sentence_transformers import models, SentenceTransformer
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [36]:
def get_text_from_article(url, xpath, rate_limiting=False):
    if rate_limiting:
        user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:89.0) Gecko/20100101 Firefox/89.0",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:92.0) Gecko/20100101 Firefox/92.0",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.64",
            "Mozilla/5.0 (Windows NT 6.1; rv:56.0) Gecko/20100101 Firefox/56.0",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Mobile Safari/537.36",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.115 Safari/537.36"
 
        ]
        headers = {"User-Agent": random.choice(user_agents)}
        req = urllib.request.Request(url, headers=headers)
        response = urllib.request.urlopen(req)
    else:
        response = urllib.request.urlopen(url)
    page_content = response.read()
    response.close()
    tree = html.fromstring(page_content)
    text = tree.xpath(xpath)
    return " ".join(text).strip()

def get_wp_articles_urls(url):
    response = urllib.request.urlopen(url)
    page_content = response.read()
    response.close()
    tree = html.fromstring(page_content)
    xpath = "//div[@class='akWTRfRe a3brEpa-']//a"
    links = tree.xpath(xpath)
    counter = 0
    urls = []
    for link in links:
        if counter % 2 == 0:
            urls.append(link.get('href'))
        counter+=1
    return urls
            
def create_files(urls, xpath, folder_name, count=1, rate_limiting=False):
    os.makedirs(folder_name, exist_ok=True)
    counter = count
    for url in urls:
        text = get_text_from_article(url, xpath, rate_limiting)
        file_name = os.path.join(folder_name,str(counter) + '.txt')
        counter+=1
        with open(file_name, 'w', encoding='utf-8') as file:
            file.write(text)

def get_text_from_wp(url, xpath, folder_name):
    try:
        urls = get_wp_articles_urls(url)
        page = 2
        while True:
            new_url = url + '/' + str(page)
            page+=1
            for url1 in get_wp_articles_urls(new_url):
                urls.append(url1)
    except Exception as e:
        create_files(urls, xpath, folder_name)

def get_bi_articles_urls(url):
    response = urllib.request.urlopen(url)
    page_content = response.read()
    response.close()
    tree = html.fromstring(page_content)
    xpath = "//main//div[@class='stream-list']//a"
    links = tree.xpath(xpath)
    urls = []
    for link in links:
        urls.append(link.get("href"))
    return urls

def get_text_from_bi(url, xpath, folder_name):
    old_size = 0
    urls = get_bi_articles_urls(url)
    new_size = len(urls)
    page = 2
    while old_size != new_size:
        new_url = url + '?page=' + str(page)
        page+=1
        old_size = new_size
        for url1 in get_bi_articles_urls(new_url):
            urls.append(url1)
        new_size = len(urls)
    create_files(urls, xpath, folder_name)

def get_kryminalki_urls(url):
    user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:89.0) Gecko/20100101 Firefox/89.0",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:92.0) Gecko/20100101 Firefox/92.0",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.64",
            "Mozilla/5.0 (Windows NT 6.1; rv:56.0) Gecko/20100101 Firefox/56.0",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Mobile Safari/537.36",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.115 Safari/537.36"
 
        ]
    headers = {"User-Agent": random.choice(user_agents)}
    req = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(req)
    page_content = response.read()
    response.close()
    tree = html.fromstring(page_content)
    xpath = "//div[@class='article-item']//a"
    links = tree.xpath(xpath)
    urls = []
    for link in links:
        urls.append('https://www.kryminalki.pl' + link.get("href"))
    return urls

def get_text_from_kryminalki(url, xpath, folder_name):
    urls = get_kryminalki_urls(url)
    for i in range(2, 200):
        time.sleep(5)
        new_url = url + '?p=' + str(i)
        for url1 in get_kryminalki_urls(new_url):
            urls.append(url1)
    create_files(urls, xpath, folder_name, 63, True)



xpath_wp = '//article//h1//text() | //article//li//text() | //article//p//text() | //article//h2//text()'
xpath_bi = "//p[@class='article_p']//text()"
xpath_kryminalki = "//h1[@class='text-break text-title']//text() | //div[@class='my-3 text-break text-lead']//text() | //div[@id='article-content']//text()"
url_przestepstwa = 'https://wiadomosci.wp.pl/tag/przest%C4%99pstwa'
url_katastrofa = 'https://wiadomosci.wp.pl/tag/katastrofa'
url_biznes = 'https://businessinsider.com.pl/biznes'
url_przestepstwa2 = 'https://www.kryminalki.pl/artykuly/45/przestepcy'
#get_text_from_wp(url_przestepstwa, xpath_wp, 'przestępstwo')
#get_text_from_wp(url_katastrofa, xpath_wp, 'katastrofa')
#get_text_from_bi(url_biznes, xpath_bi, 'biznes')
get_text_from_kryminalki(url_przestepstwa2, xpath_kryminalki, 'przestępstwo')

In [3]:
nlp = spacy.load('pl_core_news_sm')


def split_text_into_sentences(text):
    doc = nlp(text)
    return [sent.text for sent in doc.sents]

def create_files(category, ratio):
    train_counter = 1
    test_counter = 1
    for filename in os.listdir(category):
        with open(category + '/' + filename, 'r', encoding='utf-8') as file:
            text = file.read()
            sentences = split_text_into_sentences(text)
            for sentence in sentences:
                if random.randint(0, 9) > ratio:
                    path = 'dataset/train/' + category + '/' + str(train_counter) + '.txt'
                    train_counter+=1
                else:
                    path = 'dataset/test/' + category + '/' + str(test_counter) + '.txt'
                    test_counter+=1
                with open(path, 'w', encoding='utf-8') as f:
                    f.write(sentence)
                

def divide_into_test_and_trening(test_size=0.2):
    ratio = test_size * 10
    categories = ['biznes', 'katastrofa', 'przestępstwo']
    try:
        shutil.rmtree('dataset')
    except Exception as e:
        pass
    for category in categories:
        os.makedirs('dataset/test/' + category, exist_ok=True)
        os.makedirs('dataset/train/' + category, exist_ok=True)
        create_files(category, ratio)

divide_into_test_and_trening()

In [2]:

def load_data_from_folder(data_dir, label_map=None):
    texts = []
    labels = []

    if label_map is None:
        label_map = {
            d: idx
            for idx, d in enumerate(
                sorted([
                    x for x in os.listdir(data_dir)
                    if os.path.isdir(os.path.join(data_dir, x))
                ])
            )
        }

    for label, label_id in label_map.items():
        label_path = os.path.join(data_dir, label)
        if not os.path.isdir(label_path):
            continue

        for txt_file in os.listdir(label_path):
            txt_path = os.path.join(label_path, txt_file)
            if not os.path.isfile(txt_path):
                continue

            with open(txt_path, "r", encoding="utf-8") as f:
                texts.append(f.read())
                labels.append(label_id)

    return texts, labels, label_map



train_texts, train_labels, label_map = load_data_from_folder('dataset/train')
test_texts, test_labels, _ = load_data_from_folder('dataset/test')

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(label_map)

sbert = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device=device
)

with torch.no_grad():
    X_train = sbert.encode(
        train_texts,
        convert_to_tensor=True,
        batch_size=32,
        show_progress_bar=True
    )
    X_test = sbert.encode(
        test_texts,
        convert_to_tensor=True,
        batch_size=32,
        show_progress_bar=True
    )
    

{'biznes': 0, 'katastrofa': 1, 'przestępstwo': 2}


Batches:   0%|          | 0/2788 [00:00<?, ?it/s]

Batches:   0%|          | 0/1190 [00:00<?, ?it/s]

In [5]:
class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)

y_train = torch.tensor(train_labels, dtype=torch.long)
y_test  = torch.tensor(test_labels, dtype=torch.long)

X_train = X_train.clone().detach()
X_test  = X_test.clone().detach()

X_train = X_train.to(device)
X_test  = X_test.to(device)
y_train = y_train.to(device)
y_test  = y_test.to(device)

model = Classifier(
    input_dim=X_train.shape[1],
    num_classes=len(label_map)
).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
    
for epoch in range(150):
    model.train()

    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")


THRESHOLD = 0.75

model.eval()
with torch.no_grad():
    logits = model(X_test)
    probs = torch.softmax(logits, dim=1)
    max_probs, preds = torch.max(probs, dim=1)

print("\nClassification report:")
print(classification_report(test_labels, preds))

print("Confusion matrix:")
print(confusion_matrix(test_labels, preds))


Epoch 1 | Loss: 1.0947
Epoch 2 | Loss: 1.0848
Epoch 3 | Loss: 1.0749
Epoch 4 | Loss: 1.0646
Epoch 5 | Loss: 1.0538
Epoch 6 | Loss: 1.0423
Epoch 7 | Loss: 1.0298
Epoch 8 | Loss: 1.0162
Epoch 9 | Loss: 1.0015
Epoch 10 | Loss: 0.9855
Epoch 11 | Loss: 0.9682
Epoch 12 | Loss: 0.9495
Epoch 13 | Loss: 0.9292
Epoch 14 | Loss: 0.9076
Epoch 15 | Loss: 0.8845
Epoch 16 | Loss: 0.8603
Epoch 17 | Loss: 0.8347
Epoch 18 | Loss: 0.8083
Epoch 19 | Loss: 0.7805
Epoch 20 | Loss: 0.7520
Epoch 21 | Loss: 0.7232
Epoch 22 | Loss: 0.6940
Epoch 23 | Loss: 0.6650
Epoch 24 | Loss: 0.6364
Epoch 25 | Loss: 0.6079
Epoch 26 | Loss: 0.5805
Epoch 27 | Loss: 0.5542
Epoch 28 | Loss: 0.5290
Epoch 29 | Loss: 0.5060
Epoch 30 | Loss: 0.4848
Epoch 31 | Loss: 0.4651
Epoch 32 | Loss: 0.4480
Epoch 33 | Loss: 0.4327
Epoch 34 | Loss: 0.4192
Epoch 35 | Loss: 0.4078
Epoch 36 | Loss: 0.3978
Epoch 37 | Loss: 0.3895
Epoch 38 | Loss: 0.3826
Epoch 39 | Loss: 0.3768
Epoch 40 | Loss: 0.3723
Epoch 41 | Loss: 0.3681
Epoch 42 | Loss: 0.3644
E

In [6]:
torch.save(model, "pjn_model.pth")